## Visual 1: NBA vs Bulls Brand Demand Over Time (Philippines)

In [5]:
from pytrends.request import TrendReq
import pandas as pd

pytrends = TrendReq(hl='en-US', tz=360)

pytrends.build_payload(
    kw_list=["NBA"],
    geo="PH",
    timeframe="2018-01-01 2025-01-01"
)

df_nba = pytrends.interest_over_time().reset_index()
df_nba.head()


,date,NBA,isPartial
0,2018-01-01,48,False
1,2018-02-01,47,False
2,2018-03-01,49,False
3,2018-04-01,70,False
4,2018-05-01,78,False


In [6]:
df_nba = df_nba.drop(columns=["isPartial"])
df_nba.head()


,date,NBA
0,2018-01-01,48
1,2018-02-01,47
2,2018-03-01,49
3,2018-04-01,70
4,2018-05-01,78


In [11]:
import pandas as pd

# Create seasonal bands for NBA playoffs (April–June)
years = range(2018, 2025)

seasonal_bands = pd.DataFrame([
    {"start": f"{y}-04-01", "end": f"{y}-06-30"}
    for y in years
])

seasonal_bands["start"] = pd.to_datetime(seasonal_bands["start"])
seasonal_bands["end"] = pd.to_datetime(seasonal_bands["end"])


In [12]:
import altair as alt

shade = alt.Chart(seasonal_bands).mark_rect(
    fill="#CE1141",   # Bulls red, very subtle
    opacity=0.06
).encode(
    x="start:T",
    x2="end:T"
)


In [15]:
line = alt.Chart(df_nba).mark_line(
    color="black",
    strokeWidth=2
).encode(
    x=alt.X(
        "date:T",
        title="Year"
    ),
    y=alt.Y(
        "NBA:Q",
        title="Relative Search Interest (Seasonally Indexed)"
    ),
    tooltip=[
        alt.Tooltip("date:T", title="Date"),
        alt.Tooltip("NBA:Q", title="Search Interest Index")
    ]
)


In [ ]:
footnotes = pd.DataFrame({
    "text": [
        "• Google Trends is a relative index. A value of 100 marks the single month with the highest NBA search interest in the Philippines between 2018 and 2024.",
        "• Lower values mostly reflect the NBA off-season. They do not mean that interest disappears.", 
        "• The same spikes show up every year during the playoffs (marked with a red background), which suggests fan engagement is predictable and tied to the league calendar."
    ],
    "row": [0, 1, 2]
})

notes = alt.Chart(footnotes).mark_text(
    align="left",
    fontSize=11
).encode(
    x=alt.value(0),
    y=alt.Y("row:O", axis=None),
    text="text:N"
).properties(
    width=830,
    height=80  
)


final_chart = alt.vconcat(
    (shade + line).properties(
        width=830,
        height=400,
        title="Google Search Interest for the NBA in the Philippines (2018–2024)"
    ),
    notes,
    spacing=8
)

final_chart



alt.VConcatChart(...)

## Visual 2: Philippines vs Other Major NBA Markets (Fan Intensity Benchmark)

In [29]:
import pandas as pd

# Load World Bank GDP per capita data
gdp_raw = pd.read_csv(
    "data/gdp_per_capita.csv",
    skiprows=4
)

gdp_raw.head()


,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,Unnamed: 69
0,Aruba,ABW,GDP per capita (current US$),NY.GDP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,27441.550214,28440.041688,30082.158423,30645.890602,22759.807175,26749.329609,30975.998912,35718.753119,39498.594129,NaN
1,Africa Eastern and Southern,AFE,GDP per capita (current US$),NY.GDP.PCAP.CD,186.089204,186.909053,197.367547,225.400079,208.962717,226.836135,...,1334.470500,1528.104224,1552.073722,1507.085600,1351.591669,1562.416175,1679.327622,1571.449189,1615.396356,NaN
2,Afghanistan,AFG,GDP per capita (current US$),NY.GDP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,522.082216,525.469771,491.337221,496.602504,510.787063,356.496214,357.261153,413.757895,NaN,NaN
3,Africa Western and Central,AFW,GDP per capita (current US$),NY.GDP.PCAP.CD,121.936832,127.451040,133.823783,139.004980,148.545883,155.561897,...,1630.039439,1574.230564,1720.140092,2216.385055,2030.861659,2112.794076,2138.473153,1841.855064,1411.337029,NaN
4,Angola,AGO,GDP per capita (current US$),NY.GDP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2051.814621,2790.718869,2860.093648,2493.678844,1759.356199,2303.908127,3682.113151,2916.136633,2665.874448,NaN


In [60]:
countries = {
    "Philippines": "PH",
    "Indonesia": "ID",
    "Taiwan": "TW",
    "Hong Kong SAR, China": "HK",
    "Mexico": "MX",
    "Argentina": "AR",
    "Brazil": "BR",
    "Puerto Rico": "PR",
    "Colombia": "CO",
    "Peru": "PE", 
    "China": "CN",
    "Ecuador": "EC",
    "Brazil": "BR",
    "Chile": "CL",
    "Saudi Arabia": "SA",
    "Oman": "OM",
    "Egypt": "EG",
    "Ukraine": "UA",
    "Hungary": "HU",
    "Thailand": "TH",
    "Cambodia": "KH",
    "Laos": "LA",
    "Myanmar": "MM",
    "Uganda": "UG",
    "Algeria": "DZ",
    "Kazakhstan": "KZ",
    "Uzbekistan": "UZ",
    "Bolivia": "BO",
    "Paraguay": "PY",
    "Uruguay": "UY",
}


gdp = gdp_raw[gdp_raw["Country Name"].isin(countries)].copy()


In [61]:
def latest_gdp(row):
    if not pd.isna(row["2024"]):
        return row["2024"], 2024
    else:
        return row["2023"], 2023

gdp[["gdp_per_capita_usd", "gdp_year"]] = gdp.apply(
    lambda row: pd.Series(latest_gdp(row)),
    axis=1
)


In [62]:
gdp_clean = (
    gdp[["Country Name", "gdp_per_capita_usd", "gdp_year"]]
    .rename(columns={"Country Name": "country"})
    .sort_values("gdp_per_capita_usd", ascending=False)
    .reset_index(drop=True)
)

gdp_clean


,country,gdp_per_capita_usd,gdp_year
0,"Hong Kong SAR, China",54074.692852,2024.0
1,Saudi Arabia,35121.662869,2024.0
2,Uruguay,23906.513303,2024.0
3,Hungary,23292.326388,2024.0
4,Oman,20285.227290,2024.0
5,Chile,16709.889397,2024.0
6,Mexico,14185.781225,2024.0
7,Kazakhstan,14154.632496,2024.0
8,Argentina,13969.783660,2024.0
9,China,13303.148154,2024.0


Due to rate request limits, the following code has been run on a separate ssh server. The dataframe below is constructed to reflect the information.

In [1]:
# import time
# from pytrends.request import TrendReq
# import pandas as pd
# import numpy as np

# results = []

# for i, (country, geo) in enumerate(countries.items()):
#     try:
#         # Recreate session every few requests
#         if i % 5 == 0:
#             pytrends = TrendReq(hl="en-US", tz=360)

#         pytrends.build_payload(
#             kw_list=["NBA"],
#             timeframe="2018-01-01 2024-12-31",
#             geo=geo
#         )

#         data = pytrends.interest_over_time()

#         if data.empty or "NBA" not in data.columns:
#             continue

#         series = data["NBA"]
#         top10_avg = series.nlargest(max(1, int(len(series) * 0.10))).mean()

#         results.append({
#             "country": country,
#             "nba_top10_avg": top10_avg
#         })

#         # IMPORTANT: sleep between requests
#         # time.sleep(15)

#     except Exception as e:
#         print(f"Skipping {country}: {e}")
#         # time.sleep(30)  # back off harder on failure

# nba_intensity = pd.DataFrame(results)
# nba_intensity.sort_values("nba_top10_avg", ascending=False)


In [107]:
nba_intensity = pd.DataFrame({
    "country": [
        "Saudi Arabia", "Uganda", "Thailand", "Philippines", "Uzbekistan",
        "Taiwan", "Myanmar", "Oman", "Puerto Rico", "Argentina",
        "Peru", "Cambodia", "Algeria", "Hong Kong SAR, China", "Indonesia",
        "Ukraine", "Colombia", "Laos", "Mexico", "Hungary",
        "China", "Egypt", "Uruguay", "Chile", "Bolivia",
        "Ecuador", "Brazil", "Paraguay"
    ],
    "nba_top10_avg": [
        87.375, 67.375, 85.375, 84.125, 53.500,
        82.750, 62.000, 79.125, 77.875, 77.500,
        77.375, 67.375, 77.000, 76.625, 74.750,
        74.000, 73.500, 73.125, 72.875, 71.875,
        70.250, 70.250, 70.000, 67.250, 66.500,
        65.875, 65.500, 63.625
    ]
})

nba_intensity["highlight"] = nba_intensity["country"].apply(
    lambda x: "Philippines" if x == "Philippines" else "Other markets"
)


In [108]:
merged = nba_intensity.merge(
    gdp_clean,
    on="country",
    how="inner"
)

merged


,country,nba_top10_avg,highlight,gdp_per_capita_usd,gdp_year
0,Saudi Arabia,87.375,Other markets,35121.662869,2024.0
1,Uganda,67.375,Other markets,1077.912785,2024.0
2,Thailand,85.375,Other markets,7346.620221,2024.0
3,Philippines,84.125,Philippines,3984.831539,2024.0
4,Uzbekistan,53.500,Other markets,3161.700106,2024.0
5,Myanmar,62.000,Other markets,1359.050016,2024.0
6,Oman,79.125,Other markets,20285.227290,2024.0
7,Argentina,77.500,Other markets,13969.783660,2024.0
8,Peru,77.375,Other markets,8452.371671,2024.0
9,Cambodia,67.375,Other markets,2627.879698,2024.0


In [109]:
merged[["country", "gdp_per_capita_usd", "nba_top10_avg"]]


,country,gdp_per_capita_usd,nba_top10_avg
0,Saudi Arabia,35121.662869,87.375
1,Uganda,1077.912785,67.375
2,Thailand,7346.620221,85.375
3,Philippines,3984.831539,84.125
4,Uzbekistan,3161.700106,53.500
5,Myanmar,1359.050016,62.000
6,Oman,20285.227290,79.125
7,Argentina,13969.783660,77.500
8,Peru,8452.371671,77.375
9,Cambodia,2627.879698,67.375


In [110]:
merged["highlight"] = merged["country"].apply(
    lambda x: "Philippines" if x == "Philippines" else "Other markets"
)


In [120]:
import altair as alt

base = alt.Chart(merged).encode(
    x=alt.X(
        "gdp_per_capita_usd:Q",
        scale=alt.Scale(type="log"),
        title="GDP per capita (USD, log scale)"
    ),
    y=alt.Y(
        "nba_top10_avg:Q",
        title="NBA fan intensity (Top 10% months)"
    )
)

merged["label"] = merged["country"]
merged["highlight"] = merged["country"].apply(
    lambda x: "Philippines" if x == "Philippines" else "Other markets"
)


points = base.mark_circle(size=120).encode(
    color=alt.Color(
        "highlight:N",
        scale=alt.Scale(
            domain=["Philippines", "Other markets"],
            range=["#d62728", "#bdbdbd"]
        ),
        legend=None
    ),
    tooltip=[
        alt.Tooltip("country:N", title="Country"),
        alt.Tooltip("gdp_per_capita_usd:Q", title="GDP per capita", format=",.0f"),
        alt.Tooltip("nba_top10_avg:Q", title="NBA fan intensity", format=".1f")
    ]
)


labels = base.mark_text(
    align="left",
    dx=6,
    dy=-4,
    fontSize=10,
    color="#444"
).encode(
    text="label:N"
)

chart = (points + labels).properties(
    width=800,
    height=420,
    title=alt.TitleParams(
        text="NBA Fan Intensity vs Economic Capacity",
        subtitle=[
            "Asia and Latin America emerging markets",
            "Fan intensity measured using Google Trends (top 10% months, 2018–2024)"
        ]
    )
)


footnotes = pd.DataFrame({
    "text": [
        "• NBA fan intensity shows little relationship with income levels. Several lower-GDP markets display engagement comparable to much wealthier countries.",
        "• The Philippines stands out by combining high NBA engagement with low GDP per capita, suggesting fan acquisition is driven by cultural fit and habit rather than ",
        "purchasing power.",
    ],
    "row": [0, 1, 2]
})

notes = alt.Chart(footnotes).mark_text(
    align="left",
    fontSize=11
).encode(
    x=alt.value(0),
    y=alt.Y("row:O", axis=None),
    text="text:N"
).properties(
    width=800,
    height=55
)

final_chart = alt.vconcat(
    chart,   # your stacked age chart
    notes,
    spacing=8
)

final_chart


alt.VConcatChart(...)

In [ ]:
trend = base.transform_regression(
    "gdp_per_capita_usd", "nba_top10_avg"
).mark_line(color="gray", strokeDash=[4,4])

(chart + trend)


## Visual 3: Digital Readiness Funnel (Population → Internet → NBA Engagement)

In [66]:
import altair as alt
import pandas as pd

# -----------------------------
# Funnel data (millions)
# -----------------------------
funnel_data = pd.DataFrame({
    "stage": [
        "Total population",
        "Internet users",
        "Social media users"
    ],
    "users_millions": [
        115,
        97,
        90
    ]
})

funnel_chart = alt.Chart(funnel_data).mark_bar().encode(
    y=alt.Y(
        "stage:N",
        sort=list(funnel_data["stage"]),
        title=None
    ),
    x=alt.X(
        "users_millions:Q",
        title="Millions of people"
    ),
    color=alt.Color(
        "stage:N",
        legend=None,
        scale=alt.Scale(range=["#4c72b0", "#55a868", "#c44e52"])
    ),
    tooltip=[
        alt.Tooltip("stage:N", title="Stage"),
        alt.Tooltip("users_millions:Q", title="People (millions)", format=".0f")
    ]
).properties(
    width=350,
    height=280,
    title=alt.TitleParams(
        text="Philippines Digital Readiness",
        subtitle=[
            "Large, mobile-first audience enables digital-first NBA growth"
        ]
    )
)

# -----------------------------
# Sports engagement (%)
# -----------------------------
sports_data = pd.DataFrame({
    "sport": [
        "Basketball (NBA-relevant)",
        "Boxing",
        "Volleyball",
        "Football",
        "Other sports"
    ],
    "engagement_pct": [
        75,
        50,
        35,
        25,
        15
    ]
})

sports_chart = alt.Chart(sports_data).mark_bar().encode(
    y=alt.Y(
        "sport:N",
        sort="-x",
        title=None
    ),
    x=alt.X(
        "engagement_pct:Q",
        title="% of population engaged",
        scale=alt.Scale(domain=[0, 80])
    ),
    color=alt.condition(
        alt.datum.sport == "Basketball (NBA-relevant)",
        alt.value("#1f77b4"),   # calm blue highlight
        alt.value("#b0b0b0")    # neutral gray for others
    ),
    tooltip=[
        alt.Tooltip("sport:N", title="Sport"),
        alt.Tooltip("engagement_pct:Q", title="% engaged", format=".0f")
    ]
).properties(
    width=350,
    height=280,
    title=alt.TitleParams(
        text="Sports Engagement Context",
        subtitle=[
            "Basketball dominates national sports interest",
            "Estimated NBA digital audience: ~35–40 million people"
        ]
    )
)

# -----------------------------
# Combine charts
# -----------------------------
combined_chart = alt.hconcat(
    funnel_chart,
    sports_chart
).resolve_scale(
    color="independent"
)

combined_chart


alt.HConcatChart(...)

## Visual 4: Age Structure of the Philippines (Long-Term Growth Signal)

In [70]:
import altair as alt
import pandas as pd

# Philippines age structure in finer buckets (approx % shares)
age_data_split = pd.DataFrame({
    "age_group": [
        "0–14",
        "15–24",
        "25–39",
        "40–64",
        "65+"
    ],
    "percent_share": [
        31.8,
        19.5,
        22.3,
        21.7,
        4.7
    ]
})

age_chart_split = alt.Chart(age_data_split).mark_bar().encode(
    x=alt.X(
        "sum(percent_share):Q",
        stack="normalize",
        title="Population distribution (%)"
    ),
    color=alt.Color(
        "age_group:N",
        scale=alt.Scale(
            domain=["0–14", "15–24", "25–39", "40–64", "65+"],
            range=["#4c72b0", "#55a868", "#f28e2b", "#e15759", "#76b7b2"]
        ),
        title="Age group"
    ),
    tooltip=[
        alt.Tooltip("age_group:N", title="Age group"),
        alt.Tooltip("percent_share:Q", title="Share (%)", format=".1f")
    ]
).properties(
    width=600,
    height=120,
    title=alt.TitleParams(
    text="Philippines Population Age Structure (More Detailed Age Bands)",
)

)

footnotes = pd.DataFrame({
    "text": [
        "• Over 70% of the Philippine population is under age 40, creating a long runway for fan acquisition and lifetime value.",
        "• A large youth and early-adult base supports digital engagement, grassroots programs, and academy-style investments."
    ],
    "row": [0, 1]
})

notes = alt.Chart(footnotes).mark_text(
    align="left",
    fontSize=11
).encode(
    x=alt.value(0),
    y=alt.Y("row:O", axis=None),
    text="text:N"
).properties(
    width=600,
    height=55
)

final_chart = alt.vconcat(
    age_chart_split,   # your stacked age chart
    notes,
    spacing=8
)

final_chart


alt.VConcatChart(...)

## Visual 5: Basketball Ecosystem Density in Manila (Context, Not Competition)

In [ ]:
import altair as alt
import pandas as pd

# Composite index data
ecosystem_data = pd.DataFrame({
    "metric": [
        "Youth pipeline (Jr. NBA reach)",
        "Pro league demand (PBA viewership)",
        "Event hosting (FIBA World Cup)"
    ],
    "index_score": [
        40,   # 2M / 5M * 100
        35, # 975k / 5M * 100
        63    # 38k / 60k * 100
    ]
})

ecosystem_chart = alt.Chart(ecosystem_data).mark_bar().encode(
    x=alt.X(
        "index_score:Q",
        title="Ecosystem Density Index (0–100)"
    ),
    y=alt.Y(
        "metric:N",
        sort="-x",
        title=None
    ),
    color=alt.Color(
        "metric:N",
        legend=None,
        scale=alt.Scale(
            domain=[
                "Youth pipeline (Jr. NBA reach)",
                "Pro league demand (PBA viewership)",
                "Event hosting (FIBA World Cup)"
            ],
            range=["#4c72b0", "#55a868", "#c44e52"]
        )
    ),
    tooltip=[
        alt.Tooltip("metric:N", title="Ecosystem Metric"),
        alt.Tooltip("index_score:Q", title="Index (0–100)", format=".1f")
    ]
).properties(
    width=700,
    height=250,
    title=alt.TitleParams(
        text="Basketball Ecosystem Density Index — Manila & Philippines",
    )
)


footnotes = pd.DataFrame({
    "text": [
        "• Index reflects three layers of the basketball ecosystem: youth participation (Jr. NBA), sustained domestic demand (PBA viewership),",
        "  and the ability to host large international events.",
        "• Each score is normalized against realistic reference levels (e.g., typical youth program reach, domestic league audiences, and arena-scale events), not total population.",
        "• Taken together, the scores indicate basketball is not episodic in the Philippines; it is part of everyday sports consumption and long-term fan development."
    ],
    "row": [0, 1, 2, 3]
})



notes = alt.Chart(footnotes).mark_text(
    align="left",
    fontSize=11
).encode(
    x=alt.value(10),          
    y=alt.Y("row:O", axis=None),
    text="text:N"
).properties(
    width=850,              
    height=70                
)


final_chart = alt.vconcat(
    ecosystem_chart,   
    notes,
    spacing=8
)

final_chart



alt.VConcatChart(...)